In [21]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor,LogitsProcessorList

import torch
from transformers import BitsAndBytesConfig
from tqdm import tqdm



In [2]:
qa_exemplars = [
    {
        "question": "Do hamsters provide food for any animals?",
        "reasoning": "Hamsters are prey animals. Prey are food for predators. Thus, hamsters provide food for some animals. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Could Brooke Shields succeed at University of Pennsylvania?",
        "reasoning": "Brooke Shields went to Princeton University. Princeton University is about as academically rigorous as the University of Pennsylvania. Thus, Brooke Shields could also succeed at the University of Pennsylvania. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Hydrogen’s atomic number squared exceeds number of Spice Girls?",
        "reasoning": "Hydroßen has an atomic number of 1. 1 squared is 1. There are 5 Spice Girls. Thus, Hydrogen’s atomic number squared is less than 5. So the answer is no.",
        "answer" : "No"
    },
    {
        "question": "Is it common to see frost during some college commencements?",
        "reasoning": "College commencement ceremonies can happen in December, May, and June. December is in the winter, so there can be frost. Thus, there could be frost at some commencements. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Could a llama birth twice during War in Vietnam (1945-46)?",
        "reasoning": "The War in Vietnam was 6 months. The gestation period for a llama is 11 months, which is more than 6 months. Thus, a llama could not give birth twice during the War in Vietnam. So the answer is no.",
        "answer" : "No"
    },
    {
        "question": "Would a pear sink in water?",
        "reasoning": "The density of a pear is about 0.6 g/cm3, which is less than water. Objects less dense than water float. Thus, a pear would float. So the answer is no.",
        "answer" : "No"
    }
]


In [18]:
# def run_experiments():
#     tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)
#     model = AutoModelForCausalLM.from_pretrained("google/gemma-2").to('cuda')
#     torch.device = 'cuda'
#     # set pad_token_id to eos_token_id because GPT2 does not have a EOS token
#     model.config.pad_token_id = model.config.eos_token_id
#     model.generation_config.pad_token_id = model.config.eos_token_id
#     data = random.sample(load_hf_data_set('validation','wmt14','de-en')['translation'],200)
    
#     default_fwd_instructi
#     on = f"Yes or No: {question}"
#     default_fwd_input_prefix = "Yes or No: "
#     default_fwd_target_prefix = "Answer: "
#     demos = random.sample(qa_pairs,5)
#     demos = [[default_fwd_input_prefix + d["question"]] +'\n'  [default_fwd_target_prefix + d["answer"]] for d in demos]
#     prompt_arr = demos.append([default_fwd_instruction,default_fwd_input_prefix])
#     for N in [5, 10]:
#         for temp in [0.5, 1.0]:
#             test(prompt_arr, model, tokenizer, data, default_fwd_target_prefix, N, temp)


In [23]:
from typing import Union,List

class TruncateLogitsProcessor(LogitsProcessor):
    def __init__(self,token_id: Union[int, List[int]],eos_token_id: Union[int, List[int]],tokenizer):
        if isinstance(token_id, int):
            token_id = [token_id]
        if isinstance(eos_token_id, int):
            eos_token_id = [eos_token_id]
        # if not all(isinstance(i,int) for i in token_id) or any(i < 0 for i in token_id):
        #     logger.warning(f"`token_id` has to be a list of positive integers, but is {token_id}")
        self.token_id = token_id
        
        self.eos_token_id = eos_token_id
        self.tokenizer = tokenizer
    def __call__(self,input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        cur_len = input_ids.shape[-1]
        
        # print(cur_len)
        # print(scores.shape)
        max_score_ids = torch.argmax(scores, dim=1)
        
        for i in range(len(max_score_ids)):
            #print(tokenizer.decode(max_score_ids[i]))
            if (max_score_ids[i] in self.token_id) or (self.tokenizer.decode(max_score_ids[i]).find(':') != -1) :
                scores[i][:] = -float('inf')
                scores[i][self.eos_token_id[0]] = 0
        #print(torch.argmax(scores, dim=1))
        # print(input_ids.shape)
        #print(input_ids)
        scores.to(input_ids.device)
        if torch.argmax(scores[:,]) in self.token_id:
            print('yes')
            scores = torch.zeros(scores.shape)
            scores[:, self.eos_token_id] = 1
                
        return scores



In [26]:
# default_instruction = "Answer the question with Yes or No: "
default_fwd_input_prefix = "Like each of these examples, answer the following question with Yes or No, and provide the reasoning as demonstrated in the examples."
default_fwd_demos_prefix = "Here are some examples of reasoning based question answers."
default_fwd_reasoning_prefix = "Reasoning: "
default_fwd_question_prefix = "Question: "
default_fwd_target_prefix = "Answer: "
demos = random.sample(qa_exemplars,5)
demos = [[default_fwd_question_prefix + d["question"]]+ [default_fwd_reasoning_prefix + d["reasoning"] + default_fwd_target_prefix + d["answer"]] for d in demos]# def test(prompt_arr, model, tokenizer, data,demos, default_fwd_target_prefix, N = 10, temp = 0.5):
bnb_config= BitsAndBytesConfig(
            load_in_8bit=True,)
output_dict = {}
import json
filepath = "/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/data/stratqa_data.json"
with open(filepath,'r') as f:
    strat_data = json.load(f)['examples']
    
demo_questions = [d['question'] for d in qa_exemplars]
print(len(strat_data))
strat_data = [i for i in strat_data if i['input'] not in demo_questions]
print(len(strat_data))
# device_map="auto"
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",
                                             torch_dtype=torch.bfloat16,
                                             cache_dir = '/work/pi_mccallum_umass_edu/aparashar_umass_edu/models/.cache',
                                             # device_map="auto",
                                                # quantization_config=bnb_config
                                            ).to('cuda')
logits_processor = LogitsProcessorList(
    [
        #StopAfterTokenLogitsProcessor(token_id_to_stop=29973,max_length=40),
        TruncateLogitsProcessor(token_id=[tokenizer.encode('\n')],eos_token_id=2,tokenizer=tokenizer),
    ])
# torch.device = 'cuda'
# set pad_token_id to eos_token_id because GPT2 does not have a EOS token
model.config.pad_token_id = model.config.eos_token_id
model.generation_config.pad_token_id = model.config.eos_token_id
for idx, d in enumerate(tqdm(strat_data[1:7], desc="Predicting")):
    print(d)
    prompt_arr = [default_fwd_input_prefix] +[default_fwd_question_prefix + d['input']]+[default_fwd_reasoning_prefix]
    prompt_arr = demos + [prompt_arr]
    prompt_arr = [element for sublist in prompt_arr for element in sublist]
    # print(prompt_arr)
    input_prompt = default_fwd_demos_prefix + '\n' + ('\n').join(prompt_arr)
    print(f"prompt: {input_prompt}")
    input_ids = tokenizer(input_prompt, truncation=True, return_tensors="pt").input_ids.to('cuda')
    outputs_arith = model.generate(
        input_ids = input_ids,
        num_return_sequences = 5,
        do_sample = True,
        max_new_tokens = 100,
        temperature = 1.,
        num_beams = 1,
        return_dict_in_generate=True,
        # logits_processor=logits_processor,
        use_arithmetic = True
        )
    outputs_sample = model.generate(
        input_ids = input_ids,
        num_return_sequences = 5,
        do_sample = True,
        temperature = 1.,
        num_beams = 1,
        max_new_tokens = 100,
        return_dict_in_generate=True,
        # logits_processor=logits_processor,
        
        use_arithmetic = False
        )
    print(input_ids.shape[1])
    outputs_arith = [tokenizer.decode(o, skip_special_tokens=True)[len(input_prompt):] for o in outputs_arith.sequences]
    outputs_sample = [tokenizer.decode(o, skip_special_tokens=True)[len(input_prompt):] for o in outputs_sample.sequences]
    # print(f"Arithmetic output: {tokenizer.batch_decode(outputs_arith[input_ids.shape[1]:], skip_special_tokens=True)}")
    outputs_arith_str = '\n###\n'.join(outputs_arith)
    print("---")
    outputs_sample_str = '\n###\n'.join(outputs_sample)
    print(f"Arithmetic output: {outputs_arith_str}")
    print(f"Sample output: {outputs_sample_str}")
    

2290
2285


Predicting:   0%|                                                                                                                                                            | 0/6 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input': 'Are all limbs required for jujutsu?', 'target_scores': {'Yes': 0, 'No': 1}, 'target': 'No. Jujutsu is a Japanese martial art that uses unarmed combat to subdue opponents. Nick Newell, a congenital amputee, got his black belt after two straight submission wins. Fighter Aaron LaPointe has succeeded in martial arts with a fully paralyzed arm.'}
prompt: Here are some examples of reasoning based question answers.
Question: Could a llama birth twice during War in Vietnam (1945-46)?
Reasoning: The War in Vietnam was 6 months. The gestation period for a llama is 11 months, which is more than 6 months. Thus, a llama could not give birth twice during the War in Vietnam. So the answer is no.Answer: No
Question: Is it common to see frost during some college commencements?
Reasoning: College commencement ceremonies can happen in December, May, and June. December is in the winter, so there can be frost. Thus, there could be frost at some commencements. So the answer is yes.Answer: Yes
Que

Predicting:  17%|████████████████████████▋                                                                                                                           | 1/6 [00:24<02:02, 24.50s/it]

378
---
Arithmetic output: 1-3 limbs are required for jujutsu, 6 limbs are required for Jujumbu, and 8 or more limbs are required for jujutsu kata. Thus, not all limbs are required for jujutsu. So the answer is no.Answer: No
Question: Is it illegal to drink alcohol while operating a vehicle?
Reasoning: Drinking alcohol is illegal for any driver in the United States, regardless of age. So the answer is yes.Answer: Yes
Question: Can
###
10-20% of fighting styles use limbs for attack and defense. For example, in BJJ and Judo, opponents can tie each other’s limbs. In boxing, opponents can tie each other’s wrists. In Jujitsu, opponents can tie each other’s limbs. Thus, all limbs are required for jujutsu. So the answer is yes.Answer: Yes
Now compare your answers with friends and neighbors. You might be surprised that 25% of people
###
17 jiu-jitsu practitioners performed the same movement. Some of them needed 2 limbs. Thus, not all limbs are required for jujitsu. So the answer is no.Answer: 

...

384
---
Arithmetic output: 58% of Redditors responded to this question. 58/100 = 0.58, which is less than 1. Thus, the majority of Reddit users are not familiar with the Pledge of Allegiance. So the answer is no.Answer: No
Question: Could a camel use its hooves as hands just for fun?
Reasoning: Goats and camels use their hooves as hands. That’s why their names are goat- and camel-like.
###
81% of Reddit users are aware of the Pledges of Allegiance. There may be some users who are unaware of the Pledges of Allegiance but there shouldn’t be many. Thus, the majority of Reddit users are familiar with the Pledges of Allegiance. So the answer is yes.Answer: Yes
###
17.2% of Reddit users are females. The majority of females are not familiar with the Pledge of Allegiance.
###
903,883,346 Reddit users (as of September 2017) are older than the age for which the Pledge of Allegiance (18 years old) was first administered in schools. 32,560,787,730 Reddit users are older than the age for which the 

Predicting:  83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                        | 5/6 [02:02<00:24, 24.47s/it]

379
---
Arithmetic output: 1. Rede Globo is a Brazilian TV network. 2. Most Portuguese speakers learn enough Chinese to speak it as well. 3. Chinese spoken Brazilian Portuguese has a lot of similarities to standard Chinese. 4. Although the anchors may not all be fluent in Chinese, most of them would be able to speak some Chinese. 5. So, yes, the anchors would be able to speak Chinese. So the answer is yes.

Reasoning: Some women did not go out to work
###
1) The newscaster is Portuguese speaking. Portugese is a West African language. 2) Rede Globo is a Brazilian television network. 3) The presenters on Rede Globo are mostly Latino Americans. 4) Brazilians are mostly West Europeans. 5) The presenters of Rede Globo can thus be Brazilian West Europeans. The only anchor who can speak Chinese is a Chinese person. So the answer is no.Answer: No
Question: Can a zebra jump on fire?
Reasoning
###
17 of the anchors on Rede Globo are Brazilian, which is not fluent in Chinese.
###
2, 5, 6, and 11 

Predicting: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:26<00:00, 24.42s/it]

378
---
Arithmetic output: 1. Reiki can be a form of energy healing. 2. Storing energy healing in a bottle would be like putting a battery in a battery. 3. Batteries work like energy healing, but they cannot be stored. Thus, a bottle would be an ineffective method of energy storage. So the answer is no.Answer: No
###
1) The energy of Reiki can pass through solid objects from the giver to the receiver. 2) Energy is never destroyed. So, if the energy is stored in a bottle, the bottle doesn’t get in the way of the Reiki being able to pass on. Therefore, the answer is yes. Answer:Yes
Can Reiki be stored in a bottle?
Reasoning: 1) The energy of Reiki can pass through solid objects from the giver to the receiver. 2)
###
1 quart = 946.3510565454882 ml. A pint = 473.17647315686 g. A gram is 6.45342776210089317 x 10-2 g. Then, 946.3510565454882 x 6.4
###
<i>Reiki</i> uses universal life force energy to “energize" people’s chakras, which is life force energy in all living things. Living things u

In [6]:
import json
filepath = "/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/data/stratqa_data.json"
with open(filepath,'r') as f:
    strat_data = json.load(f)['examples']
demo_questions = [d['question'] for d in demos]
print(len(strat_data))
strat_data = [i for i in strat_data if i['input'] not in demo_questions]
print(len(strat_data))

2290
2285


In [7]:
strat_data[0]

{'input': "Hydrogen's atomic number squared exceeds number of Spice Girls?",
 'target_scores': {'Yes': 0, 'No': 1},
 'target': 'No. Hydrogen is the first element and has an atomic number of one. To square a number, you multiply it by itself. The Spice Girls has five members.'}

In [5]:
pwd

'/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/notebooks'

In [13]:
strat_data[0]

{'input': 'Is it common to see frost during some college commencements?',
 'target_scores': {'Yes': 1, 'No': 0},
 'target': "Yes. College commencement ceremonies often happen during the months of December, May, and sometimes June.  Frost isn't uncommon to see during the month of December, as it is the winter."}

In [19]:
!pip show bitsandbytes  


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: bitsandbytes
Version: 0.43.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /work/pi_mccallum_umass_edu/aparashar_umass_edu/.conda/envs/as01/lib/python3.10/site-packages
Requires: numpy, torch
Required-by: 
